In [3]:
!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.9 MB/s eta 0:00:00
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user


In [ ]:
import numpy as np
import time

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

rng = np.random.default_rng()

def compute_pi_cpu(n_points):
    x_rand = rng.random(n_points)
    y_rand = rng.random(n_points)

    n_inside = 0

    inside = np.sqrt(x_rand**2 + y_rand**2) <= 1.0
    n_inside = np.sum(inside)

    # for i in range(n_points):
    #    n_inside += np.sqrt(x_rand[i]**2 + y_rand[i]**2) <= 1

    pi = 4*n_inside / n_points

    return pi

In [ ]:
%%time

tic = time.time()
print(compute_pi_cpu(100000))
tac = time.time()

print("Time CPU {:f} seconds".format(tac-tic))

3.1468
Time CPU 0.012899 seconds
CPU times: user 2.62 ms, sys: 11.7 ms, total: 14.3 ms
Wall time: 13.1 ms


In [1]:
pi_kernel = """
__device__ float generateRandomNumber(long &last_draw)
{
    last_draw = last_draw*1103515245 + 12345;
    long abs = last_draw & 0x7fffffff;
    return abs / 2147483648.0;
}

__global__ void computePi(unsigned int *inside, unsigned int seed)
{
  unsigned int tid = threadIdx.x;

  long rand_seed = seed + tid;
  float x = generateRandomNumber(rand_seed);
  float y = generateRandomNumber(rand_seed);

  float r = sqrt(x*x + y*y);

  if (r <= 1.f)
  {
    inside[tid] = 1;
  }
}
"""

mod = SourceModule(pi_kernel)
func = mod.get_function("computePi")

In [2]:
def compute_pi_gpu(n_points):
  bytes_per_unit = 4

  inside_gpu = cuda.mem_alloc(bytes_per_unit * n_points)

  func(inside_gpu, np.uint32(time.time()), block=(n_points, 1, 1), grid=(1, 1, 1))

  inside_cpu = np.empty(n_points, dtype=np.uint32)
  cuda.memcpy_dtoh(inside_cpu, inside_gpu)

  n_inside = np.sum(inside_cpu)

  pi = 4 * n_inside / n_points

  return pi

In [ ]:
%%time

tic = time.time()
print(compute_pi_gpu(100000))
tac = time.time()

print("Time GPU {:f} seconds".format(tac-tic))